In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"907745c0-f89c-484b-8f8d-cdf626d98fb4\"")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

In [ ]:
from snowflake.snowpark.functions import *
from snowflake.snowpark import Session
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark.types import Variant
import snowflake.snowpark.functions as F
import json
from pprint import pprint
from sklearn.neural_network import MLPRegressor
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from snowflake.snowpark.types import FloatType
import numpy as np
#from matplotlib import pyplot as plt
from snowflake.snowpark.functions import udf
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

import hextoolkit
import getpass

#Snowflake connection info is saved in config.py
from config import snowflake_conn_prop

from snowflake.snowpark import version
print(version.VERSION)

In [ ]:
#Get our Session Object 
user_number = 'VERMAN' #PLEASE: CHANGE TO YOUR LAB NUMBER


#Build Variables
db = 'SNOWFLAKE_SAMPLE_DATA'
#schema_name = 'PUBLIC'
warehouse_name = 'COMPUTE_WH'
conn_name = 'DEMO_Snowpark'


print('Using: ' + db + ' ' + conn_name)

In [ ]:
# Create a Virtual Warehouse
hex_snowflake_conn = hextoolkit.get_data_connection(conn_name) #Note: Connection Names are Case Senstive
session = hex_snowflake_conn.get_snowpark_session()


session.sql_simplifier_enabled=True

session.sql
session.sql("USE  "+db).show()
#session.sql("USE SCHEMA "+schema_name).show()
#session.sql("USE WAREHOUSE "+warehouse_name).show()
#session.sql("CREATE OR REPLACE STAGE  snowpark_stage").show()
session.add_packages(
    *["snowflake-snowpark-python", "scikit-learn", "joblib", "xgboost", "numpy"]
)

#Test Results
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

In [ ]:
session.sql("CREATE OR REPLACE WAREHOUSE HOL_WH WITH WAREHOUSE_SIZE='X-SMALL'").collect()


In [ ]:
# Create Databse
session.sql("CREATE OR REPLACE DATABASE HOL_DB").collect()

In [ ]:
# Loading from local CSV-files
application_record_df = pd.read_csv('application_record.csv')
credit_record_df = pd.read_csv('credit_record.csv')

In [ ]:
# Upload to Snowflake
session.write_pandas(application_record_df, table_name='APPLICATION_RECORD', auto_create_table=True, overwrite=True)
session.write_pandas(credit_record_df, table_name='CREDIT_RECORD', auto_create_table=True, overwrite=True)

In [ ]:
# Creating a Pandas DataFrame
#Snowpark DataFrames vs. Pandas DataFrames
pandas_df = pd.read_csv('application_record.csv')
print(type(pandas_df))

In [ ]:
# Creating a Snowpark DataFrame
snowpark_df = session.table('APPLICATION_RECORD')
print(type(snowpark_df))

In [ ]:
# Compare size
import sys
print('Size in MB of Pandas DataFrame in Memory:\n', np.round(sys.getsizeof(pandas_df) / (1024.0**2), 2))
print('Size in MB of Snowpark DataFrame in Memory:\n', np.round(sys.getsizeof(snowpark_df) / (1024.0**2), 2))


# Converting a Snowpark DataFrame to Pandas DataFrame
#pandas_df2 = snowpark_df.to_pandas()

#pandas_df.shape, pandas_df2.shape

# Showing a Snowpark DataFrame
# snowpark_df.show() <- also possible
#snowpark_df.limit(5).to_pandas() # <- collects first 5 rows and displays as pandas-dataframe

In [ ]:
# Converting a Snowpark DataFrame to Pandas DataFrame
pandas_df2 = snowpark_df.to_pandas()

pandas_df.shape, pandas_df2.shape

# Showing a Snowpark DataFrame snowpark_df.show() <- also possible
snowpark_df.limit(5).to_pandas() # <- collects first 5 rows and displays as pandas-dataframe

In [ ]:
# Select specific columns
snowpark_df = snowpark_df.select('CODE_GENDER','NAME_INCOME_TYPE','DAYS_BIRTH',)
# snowpark_df = snowpark_df[['CODE_GENDER','NAME_INCOME_TYPE','DAYS_BIRTH']] -> pandas-like selection
snowpark_df.limit(5).to_pandas()